In [ ]:
pip install pandas numpy

In [5]:
import pandas as pd
import numpy as np

In [3]:
orders = pd.read_csv("/content/orders.csv")
products = pd.read_csv("/content/products.csv")
order_products_train = pd.read_csv("/content/order_products__train.csv")
order_products_prior = pd.read_csv("/content/order_products__prior.csv")
aisles = pd.read_csv("/content/aisles.csv")
departments = pd.read_csv("/content/departments.csv")
user_features=pd.read_pickle("/content/user_features.pkl")
product_features=pd.read_pickle("/content/product_features.pkl")
user_product_features=pd.read_pickle("/content/user_product_features.pkl")

In [6]:
cols_to_convert = ['order_number', 'order_dow', 'order_hour_of_day']

for col in cols_to_convert:
    orders[col] = orders[col].astype('Int64')

In [7]:
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


In [8]:
cols_to_convert = ['product_id', 'add_to_cart_order', 'reordered']

for col in cols_to_convert:
    order_products_train[col] = order_products_train[col].astype('Int64')

In [9]:
order_products_train

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [10]:
cols_to_convert = ['add_to_cart_order', 'reordered']

for col in cols_to_convert:
    order_products_prior[col] = order_products_prior[col].astype('Int64')

In [11]:
order_products_prior

,order_id,product_id,add_to_cart_order,reordered
0,2,33120.0,1,1
1,2,28985.0,2,1
2,2,9327.0,3,0
3,2,45918.0,4,1
4,2,30035.0,5,0
...,...,...,...,...
29670213,3129194,32707.0,3,0
29670214,3129194,47312.0,4,0
29670215,3129194,25146.0,5,0
29670216,3129194,1599.0,6,0


In [12]:
orders['days_since_prior_order'] = orders['days_since_prior_order'].fillna(0)
order_products_prior[['add_to_cart_order', 'reordered']] = order_products_prior[['add_to_cart_order', 'reordered']].fillna(0)

In [13]:
orders.isnull().sum()

,0
order_id,0
user_id,0
eval_set,0
order_number,0
order_dow,0
order_hour_of_day,0
days_since_prior_order,0


In [14]:
train_orders = orders.merge(order_products_train, on = 'order_id', how = 'inner')
train_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1187899,1,train,11,4,8,14.0,196,1,1
1,1187899,1,train,11,4,8,14.0,25133,2,1
2,1187899,1,train,11,4,8,14.0,38928,3,1
3,1187899,1,train,11,4,8,14.0,26405,4,1
4,1187899,1,train,11,4,8,14.0,39657,5,1


In [15]:
train_orders['reordered'].unique()

<IntegerArray>
[1, 0]
Length: 2, dtype: Int64

In [16]:
train_orders.drop(['eval_set', 'order_id'], axis = 1, inplace = True)

In [17]:
train_orders.head()

,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1,11,4,8,14.0,196,1,1
1,1,11,4,8,14.0,25133,2,1
2,1,11,4,8,14.0,38928,3,1
3,1,11,4,8,14.0,26405,4,1
4,1,11,4,8,14.0,39657,5,1


In [18]:
#train_users = train_orders.user_id.unique()
#train_users[:10]

In [20]:
user_product_features.shape

(13307953, 3)

In [21]:
user_product_features.head()

,user_id,product_id,user_product_reorder_count
0,1,196,9
1,1,10258,8
2,1,10326,0
3,1,12427,9
4,1,13032,2


In [ ]:
#df = user_product_features[user_product_features.user_id.isin(train_users)]
#df.head()

In [22]:
train_labels = train_orders[['user_id', 'product_id', 'reordered']]

df = train_labels.merge(user_product_features, on=['user_id', 'product_id'], how='left')

df.fillna(0, inplace=True)

print(df['reordered'].value_counts())

reordered
1    828824
0    555793
Name: count, dtype: Int64


In [23]:
df.head()

,user_id,product_id,reordered,user_product_reorder_count
0,1,196,1,9
1,1,25133,1,7
2,1,38928,1,0
3,1,26405,1,1
4,1,39657,1,0


In [25]:
df['user_product_reorder_count'].unique()

<IntegerArray>
[ 9,  7,  0,  1,  8,  2,  4,  6,  3,  5, 12, 10, 11, 29, 15, 17, 14, 16, 13,
 52, 21, 18, 20, 19, 23, 43, 33, 25, 56, 22, 35, 28, 78, 39, 34, 30, 27, 32,
 40, 38, 36, 47, 62, 46, 49, 26, 24, 48, 51, 61, 45, 44, 53, 60, 31, 58, 57,
 37, 42, 41, 50, 55, 73, 67, 70, 69, 63, 68, 66, 54, 79, 59, 85, 76, 74, 71,
 72, 64, 65, 77, 83, 80, 82, 81, 88, 91, 84, 89, 75, 94, 87, 98, 97, 92, 90,
 86, 95, 93, 96]
Length: 99, dtype: Int64

In [26]:
df.reordered.value_counts()

,count
reordered,
1,828824
0,555793


In [27]:
df.reordered.isnull().sum()

np.int64(0)

In [28]:
df.shape

(1384617, 4)

In [29]:
df.head()

,user_id,product_id,reordered,user_product_reorder_count
0,1,196,1,9
1,1,25133,1,7
2,1,38928,1,0
3,1,26405,1,1
4,1,39657,1,0


In [30]:
product_features.head()

,product_id,reorder_ratio,times_reordered
0,1,0.613391,1136
1,2,0.133333,12
2,3,0.732852,203
3,4,0.446809,147
4,5,0.6,9


In [31]:
user_features.head()

,user_id,avg_days_between_orders,total_orders
0,1,18.542373,10
1,2,14.902564,14
2,3,10.181818,12
3,4,11.944444,5
4,5,10.189189,4


In [32]:
product_features.isnull().sum()

,0
product_id,0
reorder_ratio,0
times_reordered,0


In [33]:
df = df.merge(product_features, on='product_id', how='left')
df = df.merge(user_features, on='user_id', how='left')

df.fillna(0, inplace=True)

df = df.astype({
    'user_product_reorder_count': 'int64',
    'times_reordered': 'int64',
    'total_orders': 'int64'
})

In [34]:
df.head()

,user_id,product_id,reordered,user_product_reorder_count,reorder_ratio,times_reordered,avg_days_between_orders,total_orders
0,1,196,1,9,0.77648,27791,18.542373,10
1,1,25133,1,7,0.740155,4586,18.542373,10
2,1,38928,1,0,0.827769,10958,18.542373,10
3,1,26405,1,1,0.441516,536,18.542373,10
4,1,39657,1,0,0.766288,3846,18.542373,10


In [37]:
df['total_orders'].unique()

array([10, 14,  4, 20,  3,  5, 12, 13, 40,  6, 33, 18, 81,  8, 23, 16, 11,
       19, 67, 27,  7, 39, 24,  9, 17, 71, 15, 26, 32, 76, 45, 31, 22, 44,
       58, 28, 99, 70, 49, 21, 29, 68, 41, 38, 42, 25, 63, 86, 52, 54, 35,
       36, 34, 51, 30, 94, 56, 78, 75, 72, 59, 69, 37, 43, 83, 46, 85, 50,
       47, 74, 55, 77, 61, 53, 48, 79, 66, 73, 57, 60, 82, 97, 89, 84, 62,
       88, 64, 90, 91, 95, 65, 80, 87, 93, 98, 96, 92])

In [36]:
df.shape

(1384617, 8)

In [38]:
df.isnull().sum()

,0
user_id,0
product_id,0
reordered,0
user_product_reorder_count,0
reorder_ratio,0
times_reordered,0
avg_days_between_orders,0
total_orders,0


In [39]:
df.to_pickle('final_data.pkl')

In [40]:
Final = pd.read_pickle('final_data.pkl')
Final.head()

,user_id,product_id,reordered,user_product_reorder_count,reorder_ratio,times_reordered,avg_days_between_orders,total_orders
0,1,196,1,9,0.77648,27791,18.542373,10
1,1,25133,1,7,0.740155,4586,18.542373,10
2,1,38928,1,0,0.827769,10958,18.542373,10
3,1,26405,1,1,0.441516,536,18.542373,10
4,1,39657,1,0,0.766288,3846,18.542373,10


In [41]:
Final['total_orders'].unique().sum()

np.int64(4947)